# Create a 2D Surface Grain Boundary in a film

Use commensurate lattice matching algorithm to create grain boundaries in 2D materials by finding appropriate twist angles between two orientations of the same material.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete (depending on the parameters can take a few min).
1. Scroll down to view results.

## Notes

1. We perform commensurate lattice matching to find valid supercells that achieve the desired twist angle.
1. When the matching is finished, grain boundaries with angles close to the target are presented.
1. The algorithm searches for supercell matrices within specified size limits.
2. The two orientations are placed next to each other in the x-direction with a gap in between.
1. Atoms on the edge of the left orientation are handled to overlap with the right orientation in the interfacial region.
1. For more information, see the [Introduction](Introduction.ipynb) notebook.


## 1. Prepare the Environment
### 1.1. Set up the notebook
Set the following flags to control the notebook behavior
For more information on the parameters and algorithm, refer to [Grain Boundary Builder Source](https://github.com/Exabyte-io/made/blob/35b9f318f5d667e0f5af023f3178bc4404317ab0/src/py/mat3ra/made/tools/build/grain_boundary/builders.py#L103)
`EDGE_INCLUSION_TOLERANCE` is a fine-tuning parameter that controls the inclusion of the edge atoms for both orientations in the gap.
For example of Graphene at 17.9 degrees: orange and green atoms are present with the value of 0.5 Angstroms, with value of 0, they will not be included.

<img src="https://objects.mat3ra.com/images/notebooks/grain_boundary_edge_inclusion_tolerance.png" alt="Edge Inclusion Tolerance" width="400"/>


In [ ]:
# Material selection
MATERIAL_INDEX = 0  # Index in the list of materials

# Grain boundary parameters
TARGET_TWIST_ANGLE = 17.9  # in degrees
BOUNDARY_GAP = 2.0  # Gap between two orientations in X direction, in Angstroms
XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]] # Supercell matrix to be applied to each of the orientations before matching
MILLER_INDICES = (0, 0, 1)  # Miller indices for the supercell matching
VACUUM = 10.0  # Vacuum thickness in Angstroms, added to the top and bottom of the grain boundary

# Search algorithm parameters
MAX_REPETITION = 6  # Maximum supercell matrix element value
ANGLE_TOLERANCE = 2.5  # in degrees
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# Distance tolerance for two atoms to be considered too close. 
# Used when merging two orientations to remove the atoms of the first one. 
# Should be less than the expected bond length
DISTANCE_TOLERANCE = 1.2  # in Angstroms

# How much to expand inclusion of the edge atoms for both orientations and fill in the gap region.
# A fine-tuning parameter
EDGE_INCLUSION_TOLERANCE = 0.5  # in Angstroms

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
CELL_REPETITIONS_FOR_VISUALIZATION = [3, 3, 1]

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+g03efa43-py3-none-any.whl", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("")

### 1.3. Load and preview input material

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

## 2. Prepare Material
### 2.1. Select and visualize initial material

In [ ]:
from utils.visualize import visualize_materials

material = materials[MATERIAL_INDEX]

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(material, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)

## 3. Generate Surface Grain Boundary


In [ ]:
from mat3ra.esse.models.core.reusable.axis_enum import AxisEnum
from mat3ra.made.tools.helpers import create_grain_boundary_linear


grain_boundary = create_grain_boundary_linear(
    material=material,
    target_angle=TARGET_TWIST_ANGLE,
    angle_tolerance=EDGE_INCLUSION_TOLERANCE,
    max_repetition_int=MAX_REPETITION,
    direction=AxisEnum.x,
    gap=BOUNDARY_GAP,
    miller_indices=MILLER_INDICES,
    vacuum=VACUUM
)


## 4. Preview grain boundary

In [ ]:
actual_angle = grain_boundary.metadata.build[-1].configuration.get("actual_angle")
print(actual_angle)
print(f"Target angle: {TARGET_TWIST_ANGLE}°")
print(f"Actual angle: {actual_angle}°")
print(f"Number of atoms: {len(grain_boundary.basis.elements.ids)}")

visualize_materials(grain_boundary, repetitions=[1, 1, 1])
visualize_materials(grain_boundary, repetitions=[1, 1, 1], rotation="-90x")

### 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials

set_materials(grain_boundary)